# HOMEWORK 4

For this homework you are going to implement a lane line detector. Lane line detection is crucial for ADAS (Advanced Driver Assistance Systems) systems and, in particular, for LKA (Lane Keep Assist). You will use a [picture](https://en.wikipedia.org/wiki/Lane_departure_warning_system) from a front facing camera (mounted on the car) and will implement the following steps:
* Convert image to gray scale
* Compute edge map
* Apply Hough transform to obtain line parametrizations

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
from showlist import show_image_list
from drow import draw_lines
plt.rcParams['figure.figsize'] = [15, 10]

Let's load and show the camera frame.

In [ ]:
img = cv2.imread('data/dashcam.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, None, fx=0.5, fy=0.5)
plt.imshow(img)

In [ ]:
# Convert image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
# Obtain edge map
# Hint: you can use Canny edge detector with th_low = 100, th_high = 150
edges = cv2.Canny(gray, threshold1 = 100, threshold2 = 150)

edges2 = cv2.Canny(gray, threshold1 = 100, threshold2 = 200)
# We are only interseted in the road so we will remove everything above the horizon
edges[0:330] = 0
edges2[0:330] = 0

In [ ]:
# Let's plot the images
images = [img, gray, edges, edges2]
lables = ['Original', 'Gray', 'Edge map 100-150', 'Edge map 100-200']

show_image_list(images, lables, grid=False)

In [ ]:
# Apply Hough transform to parametrize the lines
# Hint 1: Offset resolution of 2 pixels and slope resolution of 2 degrees work well in this case
# Hint 2: A suitable value for the accumulator threshold is 190
lines = cv2.HoughLines(edges, rho=2, theta=np.deg2rad(2), threshold=200)
# Let's get rid of the unnecessary dimension
lines = lines[:, 0, :]

result_all_lines = np.copy(img)

draw_lines(result_all_lines, lines)

show_image_list([edges,result_all_lines],['Edge map','Hough lines'],grid=False)

In [ ]:
# Plot the resulting Hough lines
result_all_lines = np.copy(img)

draw_lines(result_all_lines, lines)

show_image_list([edges,result_all_lines],['Edge map','Hough lines'],grid=False)

The edge map looks good but the Hough lines are too noisy. Let's clean the Hough lines first by removing all lines that we know cannot represent a lane line. In other words, all lines that are approximately horizontal shall be removed. Remember that horizontal lines correspond to theta = 90 degrees.

In [ ]:
# Filter out all lines that are approximately horizontal (+/- 20 degrees). 
from_theta = np.deg2rad(90 - 20)
to_theta = np.deg2rad(90 + 20)

filtered_lines = []
for line in lines:
    # Extract theta for current line (remember Hough works with radians)
    theta = line[1]
    # Keep line if theta is not horizontal
    if theta < from_theta or theta > to_theta :
        filtered_lines.append(line)

In [ ]:
# Let's plot the resulting filtered lines
result_filtered_lines = np.copy(img)
 
draw_lines(result_filtered_lines, filtered_lines)

show_image_list([result_all_lines, result_filtered_lines], ['Hough lines all', 'Hough lines filtered'], grid=False)

The result is now much better, but still we see some very similar lines. How can we get rid of them?
* Let's apply k-means clustering. It will find the clusters of the 6 we see in the picture lines and use the averages.

In [ ]:
# We will apply k-means clustering to refine the detected lines.
# Don't worry, we will learn about the clustering later in the course :-)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6).fit(filtered_lines)
kmeans.cluster_centers_

In [ ]:
# Again, let's plot the resulting filtered lines
result_k_means = np.copy(img)

draw_lines(result_k_means, kmeans.cluster_centers_)

show_image_list([result_filtered_lines, result_k_means], ['Hough lines filtered by degrees', 'Hough lines filtered by k-means clustering'], grid=False)

### Questions
* Do you see anything strange in the final result?

    In the processed area, the lines coincide with the road markings quite accurately. K-means clustering significantly improved the result. The resulting lines go through the entire image.

* Do you think the Hough transform resolution is important for obtaining a good result? Why?

    Yes. Choosing the values of resolution parameters for Hough transform, we can get more lines in general and lines close to each other, which will require additional fitting of the results or not enough lines for further correct processing of the result.

* Do you think the Hough transform accumulator threshold is important for obtaining a good result? Why?

    Yes. Accumulator threshold allows you to filter out lines that do not cross the required number of pixels, which can significantly reduce the number of lines in the output and provide a more suitable set of lines for further processing